In [1]:
import pandas as pd
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from datetime import timedelta

# โหลดข้อมูล
DATA_PATH = os.path.join("..", "DataSet", "Air_quality_cleaned_v1.csv")
df = pd.read_csv(DATA_PATH)

# เลือกคอลัมน์
df = df[['Date', 'Time', 'Temp', 'Humidity','PM2.5', 'VOC', 'CO2', 'HCHO']]

# จัดการ Missing Values
numeric_cols = df.select_dtypes(include=np.number).columns
df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

# โหลดโมเดลและ Scaler
rf = joblib.load("random_forest_model.pkl")
scaler = joblib.load("Radom.pkl")

# แปลงคอลัมน์ 'Date' เป็น datetime
df['Date'] = pd.to_datetime(df['Date'])

# คำนวณค่าเฉลี่ยรายวัน
df_daily = df.groupby(df['Date'].dt.date)[['PM2.5', 'VOC', 'Temp', 'Humidity', 'HCHO']].mean()

# เลือก 7 วันล่าสุด
last_7_days = df_daily.tail(7)

# คำนวณค่าเฉลี่ยของ 7 วันล่าสุด
average_last_7_days = last_7_days.mean()

# สร้าง DataFrame สำหรับ 7 วันข้างหน้า
current_date = df['Date'].dt.date.max()
future_dates = [current_date + timedelta(days=i) for i in range(1, 8)]
future_df = pd.DataFrame([average_last_7_days.values] * 7, index=future_dates, columns=average_last_7_days.index)

# Scaling ข้อมูล 7 วันข้างหน้า
future_scaled = scaler.transform(future_df)

# ทำนายค่า CO2 สำหรับ 7 วันข้างหน้า
predicted_co2_future = rf.predict(future_scaled)

# แปลงวันที่เป็นรูปแบบ DD-MM-YYYY
future_dates_formatted = [date.strftime('%d-%m-%Y') for date in future_dates]

# แสดงผลการทำนายและวันที่
print("ค่า CO2 ที่ทำนายสำหรับ 7 วันข้างหน้า (ค่าเฉลี่ยรายวัน):")
for i, (date, prediction) in enumerate(zip(future_dates_formatted, predicted_co2_future)):
    print(f"วันที่ {date}: {prediction:.2f}")

ค่า CO2 ที่ทำนายสำหรับ 7 วันข้างหน้า (ค่าเฉลี่ยรายวัน):
วันที่ 13-03-2025: 411.53
วันที่ 14-03-2025: 411.53
วันที่ 15-03-2025: 411.53
วันที่ 16-03-2025: 411.53
วันที่ 17-03-2025: 411.53
วันที่ 18-03-2025: 411.53
วันที่ 19-03-2025: 411.53
